In [ ]:
# emo – emotion

# czym jest polemo_out, in i official? Czy mają część wspólną. Jeśli tak, czy etykiety się zgadzają?
# Czym różni się zbiór "hotel_text" od "hotel_sentences" w polemo official? Mają część wspólną?
# Czy ocena poszczególnych słów w aspect emo jest taka sama dla różnych opinii? Czy są słowa, które nie mają oceny w żadnej opiniii?
# Czy jesteśmy w stanie wykorzystać aspect emo? Uśredniać? Samemu generować tylko oceny dla słów? Wziąć oceny dla słów, które będą naszymi kandydatami na keywords?

In [ ]:
import pandas as pd
import numpy as np
from pprint import pprint
from datasets import load_dataset

## Read data

In [ ]:
polemo_offi_config = pd.read_json('data/polemo2-official/dataset_infos.json') #this is config file
polemo_category = "hotels_text"
polemo_official = load_dataset("data/polemo2-official/", polemo_category) # only oppinions about hotels
df_polemo_official = pd.DataFrame(polemo_official["train"])

aspectemo = load_dataset("data/aspectemo")
df_aspectemo = pd.DataFrame(aspectemo["train"])

df_opta_reviews = pd.read_json("data/OPTA-treebank-reviews/OPTA-treebank-0.1.json")
df_opta_skladnica = pd.read_json("data/OPTA-treebank-skladnica/skladnica_output.json")

## Explore data

### Polemo official

Polemo out and polemo in includes in polemo official, so we don't use them.

In [ ]:
# Available categories of oppinions in polemo
list(polemo_offi_config.columns)

In [ ]:
df_polemo_official.head()

In [ ]:
df_polemo_official.target.value_counts()

In [ ]:
# get meanings of labels in polemo_official based on polemo_in (polemo out has the same meanings)
df_polemo_out = pd.read_csv("data/klej_polemo2_in/train.csv")

df_polemo_out.rename({'sentence': 'text'}, axis=1, inplace=True)
df_intersection = pd.merge(df_polemo_out, df_polemo_official, how='inner', on=['text'])
df_intersection.head()

df_intersection.drop("text", axis=1, inplace = True)
df_intersection.drop_duplicates("target_y", inplace = True)

df_intersection

### Aspectemo

In [ ]:
print(df_aspectemo.head(3))
# ["O", "a_minus_m", "a_minus_s", "a_zero", "a_plus_s", "a_plus_m", "a_amb"]

example = 2
list(zip(df_aspectemo.iloc[example].tokens, df_aspectemo.iloc[example].labels))

### OPTA reviews

The file contains JSON formatted results of annotation of opinions and their targets. 

Each item in the list is a sentence which contains following fields:

'file_id': ID of review (used internally, same sentence may appear multiple times with different sentiments and opinin target pairs) </br>
'domain': review type (perfume or clothes) </br>
'dist':  dependency path distance between S and T, filled only for the 1st batch of annotations </br>
'isSentIncorrect': human annotation - is sentiment word S incorrect? </br>
'isAttrIncorrect': human annotation - is opinion target word T incorrect? </br>
'parsedSent': CONLL-formatted parsed sentence; last column contains pointers to : </br>
	S = sentiment word </br>
	T = opinion target word </br>
'isStrError': human annotation - is dependency structure erroneous between S and T </br>
'isAtrRelToSent': human annotation - is S related to T  </br>
'rule_id': ID of extraction rule that pointed to T (see "extraction rules")

read about CONLL format here: https://universaldependencies.org/format.html

In [ ]:
df_opta_reviews.head()

In [ ]:
def conll_get_word_and_misc(conll: list) -> list:
    # based on readme and https://universaldependencies.org/format.html
    conll = conll[0].replace("\n", "")
    conll_list = conll.split("\t")[:-1] # last field is '\n'
    return [conll_list[1]] + [conll_list[-1]]

def df_from_parsedSent(df: pd.DataFrame):
    opta_words = df["parsedSent"].apply(conll_get_word_and_misc)
    opta_words = list(zip(*opta_words))
    df_opta_words = pd.DataFrame()
    df_opta_words["word"] = opta_words[0]
    df_opta_words["misc"] = opta_words[1]
    return df_opta_words

print("Example od parsedSent\n", df_opta_reviews["parsedSent"][0][0])

df_opta_words = df_from_parsedSent(df_opta_reviews)
print("Words with their labels(?)\n", df_opta_words.head(), "\n")

print("Class distribution for labels(?)\n", df_opta_words["misc"].value_counts())

# I don't quite understand how this dataset works :(


### OPTA składnica

In [ ]:
df_opta_skladnica.head()

In [ ]:
df_opta_words = df_from_parsedSent(df_opta_skladnica)
print(df_opta_words.head())
df_opta_words["misc"].value_counts()

# Looks familiar to OPTA reviews, there are many _ in last column from parsedSent. I don't think it can be useful for us
